In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
os.environ['HUGGINGFACE_API_KEY'] = os.getenv('HUGGINGFACE_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

## Search Engine Tools

In [ ]:
from langchain_community.tools          import ArxivQueryRun, WikipediaQueryRun, Tool
from langchain_community.utilities      import WikipediaAPIWrapper, ArxivAPIWrapper, DuckDuckGoSearchAPIWrapper

## Custom Tools with Wikipedia and Arxiv

In [ ]:
wikiWrapperAPI = WikipediaAPIWrapper(
    top_k_results = 1, 
    doc_content_chars_max = 250
)

wiki = WikipediaQueryRun(api_wrapper = wikiWrapperAPI)

In [ ]:
wiki.name

In [ ]:
arxivAPIWrapper = ArxivAPIWrapper(
    top_k_results = 1,
    doc_content_chars_max = 250
)

arxiv = ArxivQueryRun(api_wrapper = arxivAPIWrapper)

In [ ]:
arxiv.name

In [ ]:
search = DuckDuckGoSearchAPIWrapper()
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Web search via DuckDuckGo"
)


In [ ]:
tools = [search_tool, wiki, arxiv]


## Custom RAG Tools

In [ ]:
from langchain_community.document_loaders       import WebBaseLoader
from langchain_community.vectorstores           import FAISS
from langchain_openai                           import OpenAIEmbeddings
from langchain_text_splitters                   import RecursiveCharacterTextSplitter

In [ ]:
webBasedLoader = WebBaseLoader("https://docs.smith.langchain.com/")

In [ ]:
docs = webBasedLoader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)

In [ ]:
vectorStoreDB = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
retriever = vectorStoreDB.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever          import create_retriever_tool

retrieverTool = create_retriever_tool(
    retriever,
    "langsmith-search",
    "Search any information about Langsmith"
)

retrieverTool.name

In [ ]:
tools = [search_tool, wiki, retrieverTool] 
tools


In [ ]:
from langchain_groq     import ChatGroq

model_name = os.getenv("GROQ_MODEL", "llama-3.1-8b-instant")
llm = ChatGroq(groq_api_key = groq_api_key, model_name = model_name)

## Prompt Template

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

## Agents

In [ ]:
from langchain.agents   import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

In [ ]:
## Agent Executor

from langchain.agents import AgentExecutor

agentExecutor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agentExecutor

In [ ]:
agentExecutor.invoke({"input": "Tell me about Langsmith"})

In [ ]:
agentExecutor.invoke({"input": "What is Machine Learning"})

In [ ]:
agentExecutor.invoke({"input": "What is the paper 1706.03762 about?"})